In [1]:
from yuning_util.dev_mode import DevMode

dev_mode = DevMode('k-seq')
dev_mode.on()
from importlib import reload


In [2]:
from k_seq.data.seq_table import SeqTable
from k_seq.estimator import least_square

byo_doped = SeqTable.load_default_dataset('byo-doped')

Load BYO-doped pool data from pickled record from /mnt/storage/projects/k-seq/datasets/byo_doped.pkl
Imported!


In [3]:
seq_table = byo_doped.table_in_all_samples[:100]
seq_table

,A1,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,D3,E1,E2,E3
CTACTTCAAACAATCGGTCTG,0.270427,0.369641,0.598627,0.378180,0.400688,0.378988,0.153022,0.129371,0.154642,0.030471,0.029956,0.029584,0.006454,0.006420,0.005430
CCACACTTCAAGCAATCGGTC,0.291469,0.399418,0.667322,0.371948,0.382656,0.371522,0.132896,0.110417,0.128007,0.025532,0.025574,0.023251,0.005178,0.005254,0.004230
ATTACCCTGGTCATCGAGTGA,0.209456,0.241876,0.407345,0.306432,0.273955,0.281231,0.292973,0.159222,0.217357,0.071694,0.059787,0.073648,0.019961,0.019851,0.020001
AAGTTTGCTAATAGTCGCAAG,0.131800,0.144669,0.250802,0.173294,0.136528,0.134145,0.064863,0.037381,0.051900,0.012268,0.008453,0.009117,0.002671,0.001447,0.001554
GTACTTCAAACAATCGGTCTG,0.216827,0.263438,0.433435,0.268581,0.265559,0.248443,0.099804,0.080341,0.092561,0.019440,0.017776,0.016141,0.003888,0.003423,0.002846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATTACCTTGGTCATCGAGTGA,0.048949,0.055018,0.086575,0.031809,0.027479,0.026645,0.027071,0.012715,0.017160,0.007252,0.005843,0.007162,0.002381,0.001702,0.001751
AAGGTTGCTAATAGTCGCAAG,0.078331,0.063535,0.103274,0.034626,0.023046,0.023729,0.009061,0.004474,0.006252,0.001856,0.001256,0.001741,0.000710,0.000233,0.000246
AAGTTTGCTAATAGTCGTAAG,0.061580,0.086690,0.163359,0.132067,0.106216,0.103799,0.060309,0.035236,0.050934,0.011256,0.007557,0.008326,0.002504,0.001393,0.001533
ATTACCCTGGTCATCGAGGGA,0.135785,0.134430,0.215102,0.095882,0.082842,0.082242,0.025199,0.017149,0.023081,0.004949,0.004217,0.005021,0.001313,0.001005,0.001059


In [66]:
from k_seq.model.kinetic import BYOModel
reload(least_square)
import numpy as np



def kA(param):
    return param[0] * param[1]

batch_fitter = least_square.BatchFitter(
    table=seq_table,
    x_data=byo_doped.x_values,
    bounds=[[0, 0], [np.inf, 1]],
    metrics={'kA': kA},
    model=BYOModel.func_react_frac_no_slope,
    seq_to_fit=None,
    bootstrap_num=10,
    bs_record_num=10,
    bs_method='pct_res',
    keep_single_fitters=True
)

batch_fitter.fit(deduplicate=True, parallel_cores=6)


In [50]:
batch_fitter.save_model(model_path='/mnt/storage/projects/ls_test_model.pkl',
                        result_path='/mnt/storage/projects/ls_test_result.pkl',
                        table_path='/mnt/storage/projects/ls_test_table.pkl')

In [67]:
batch_fitter_restored = least_square.BatchFitter.load_model(
    model_path='/mnt/storage/projects/ls_test_model.pkl',
    result_path='/mnt/storage/projects/ls_test_result.pkl',
    table_path='/mnt/storage/projects/ls_test_table.pkl'
)

In [69]:
batch_fitter_restored.summary()

,k,A,kA,k_mean,k_std,k_2.5%,k_50%,k_97.5%,A_mean,A_std,A_2.5%,A_50%,A_97.5%,kA_mean,kA_std,kA_2.5%,kA_50%,kA_97.5%
CTACTTCAAACAATCGGTCTG,209.577289,0.419282,87.871897,227.793577,30.101925,183.527569,235.504995,272.880957,0.371537,0.019456,0.342960,0.369122,0.400581,84.290056,8.833241,69.362125,83.865444,96.721620
CCACACTTCAAGCAATCGGTC,155.360535,0.455281,70.732707,191.335286,31.255964,150.357655,179.369763,230.061962,0.398145,0.046166,0.338217,0.398662,0.482177,75.224648,8.224023,63.810704,73.157825,88.703639
ATTACCCTGGTCATCGAGTGA,679.435990,0.287239,195.160175,675.111101,154.655423,436.847766,694.374626,906.127708,0.292542,0.025218,0.257264,0.290320,0.335522,195.146296,36.218669,140.332297,207.961203,238.794231
AAGTTTGCTAATAGTCGCAAG,164.256810,0.176522,28.994937,162.359705,43.133003,92.139355,167.157756,224.756763,0.153621,0.023995,0.120924,0.143998,0.187280,24.257060,4.944755,16.866316,23.400773,31.824646
GTACTTCAAACAATCGGTCTG,169.874808,0.306387,52.047490,173.986857,35.610925,132.917610,170.081566,235.983283,0.280397,0.023706,0.246343,0.280054,0.314236,48.375537,8.004349,37.064492,49.374640,59.055572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATTACCTTGGTCATCGAGTGA,66.026812,0.064682,4.270720,80.921904,63.205100,10.702282,72.241450,171.257073,0.262197,0.189313,0.083684,0.198385,0.625755,12.338670,5.214166,5.636559,12.284398,20.788731
AAGGTTGCTAATAGTCGCAAG,27.909695,0.105049,2.931883,28.750158,17.650928,6.723354,26.343017,57.471269,0.168387,0.113961,0.078620,0.116370,0.400353,3.442496,0.982127,2.387182,3.198692,5.232970
AAGTTTGCTAATAGTCGTAAG,287.678142,0.110497,31.787515,262.049762,138.475156,109.756936,239.722833,501.356263,0.108131,0.014309,0.086123,0.111321,0.126621,26.792155,10.748826,12.823188,27.562664,43.011452
ATTACCCTGGTCATCGAGGGA,68.464446,0.165980,11.363719,66.097465,13.276268,47.065035,64.996524,88.636856,0.172636,0.016474,0.148694,0.174595,0.199087,11.234127,1.367318,9.324052,11.293466,13.448596


## Runtime test

In [48]:
# load pickled point estimation fitter object from fitting
from k_seq.utility.file_tools import read_pickle

point_est = read_pickle('/mnt/storage/projects/k-seq/working/byo_doped/least_square/pod_run_10-19/fitter_bs0_mpct_res_c40.pkl')

In [11]:
%%timeit
# load json results as an whole string

import json

with open('/mnt/storage/projects/k-seq/working/byo_doped/least_square/pod_run_10-19/fitter_bs0_mpct_res_c40_converted_from_pickle.json', 'r') as handle:
    point_est_json = json.load(handle)

1.48 s ± 4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
# load json results as an whole string

import json

with open('/mnt/storage/projects/k-seq/working/byo_doped/least_square/pod_run_10-19/fitter_bs500_mdata_c40_converted_from_pickle.json', 'r') as handle:
    ci_est_json = json.load(handle)

4.92 s ± 6.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
point_est

In [86]:
file_tools.dump_pickle(point_est_json, path='/mnt/storage/projects/k-seq/working/byo_doped/least_square/pod_run_10-19/fitter_bs0_mpct_res_c40_converted_from_pickle.pkl')

In [88]:
%timeit point_est_json_pkled = file_tools.read_pickle('/mnt/storage/projects/k-seq/working/byo_doped/least_square/pod_run_10-19/fitter_bs0_mpct_res_c40_converted_from_pickle.pkl')

328 ms ± 6.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Conclusion: a fitting version-independent pickle saver for model should be good!

Save model and fitting results independently

In [89]:
point_est.table

,A1,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,D3,E1,E2,E3
CTACTTCAAACAATCGGTCTG,0.270427,0.369641,0.598627,0.378180,0.400688,0.378988,0.153022,0.129371,0.154642,0.030471,0.029956,0.029584,0.006454,0.006420,0.005430
CCACACTTCAAGCAATCGGTC,0.291469,0.399418,0.667322,0.371948,0.382656,0.371522,0.132896,0.110417,0.128007,0.025532,0.025574,0.023251,0.005178,0.005254,0.004230
ATTACCCTGGTCATCGAGTGA,0.209456,0.241876,0.407345,0.306432,0.273955,0.281231,0.292973,0.159222,0.217357,0.071694,0.059787,0.073648,0.019961,0.019851,0.020001
AAGTTTGCTAATAGTCGCAAG,0.131800,0.144669,0.250802,0.173294,0.136528,0.134145,0.064863,0.037381,0.051900,0.012268,0.008453,0.009117,0.002671,0.001447,0.001554
GTACTTCAAACAATCGGTCTG,0.216827,0.263438,0.433435,0.268581,0.265559,0.248443,0.099804,0.080341,0.092561,0.019440,0.017776,0.016141,0.003888,0.003423,0.002846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCTCACTTCAAGCCATTGATC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002568
AAGTTTGCTAATGGCTGCAAA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002568
CCACACCTCAACCAATCTGCC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005136
CGACCTCAAACAACCGGTCAG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002568


## Load model

In [20]:
import numpy as np
from k_seq.data import seq_table
reload(seq_table)
SeqTable = seq_table.SeqTable
from k_seq.model import kinetic
reload(kinetic)
BYOModel = kinetic.BYOModel
from k_seq.utility import func_tools
reload(func_tools)
from k_seq.estimator import least_square
BatchFitter = least_square.BatchFitter


fit_partial = -1
bootstrap_num = 0
bs_return_num = 0
bs_method = 'pct_res'

seq_table = SeqTable.load_default_dataset('byo-doped')


if fit_partial > 0:
    seq_test = seq_table.reacted_frac_filtered.index.values[:int(fit_partial)]
else:
    seq_test = None

batch_fitter = BatchFitter(
    table=seq_table.reacted_frac_filtered, x_values=seq_table.x_values, bounds=[[0, 0], [np.inf, 1]],
    model=BYOModel.func_react_frac_no_slope, seq_to_fit=seq_test,
    bootstrap_num=bootstrap_num, bs_return_num=bs_return_num, bs_method=bs_method
)

Load BYO-doped pool data from pickled record at /mnt/storage/projects/k-seq/datasets/byo_doped.pkl
Imported!


In [26]:
batch_fitter.fit(dry_run=True)

Exception: Can not create fitting worker for ATTACCCCTGTCATCTAGTGA

841431

In [68]:
class SingleFitter():
    """Class to fit a single kinetic model for one sequence

    Attributes:

    """

    def __repr__(self):
        return f"Single fitter for {self.name}"\
               f"<{self.__class__.__module__}{self.__class__.__name__} at {hex(id(self))}>"

    def __str__(self):
        return f"Single fitter for {self.name}"

    def __init__(self, x_data, y_data, model, name=None, parameters=None, weights=None, bounds=None, opt_method='trf',
                 bootstrap_num=0, bs_return_num=None, bs_method='pct_res',
                 exclude_zero=False, init_guess=None, metrics=None, rnd_seed=None, **kwargs):
        """

        Args:
            x_data (list, np.ndarray, pd.Series): list object will convert to np.ndarray
            y_data (list, np.ndarray, pd.Series):
            model:
            name:
            weights:
            bounds:
            opt_method:
            bootstrap_num:
            bs_return_num:
            bs_method:
            exclude_zero:
            init_guess:
            metrics:
            rnd_seed:
            **kwargs:
        """
        import numpy as np
        from k_seq.utility.func_tools import DictToAttr, get_func_params


#         if len(x_data) != len(y_data):
#             raise ValueError('Shapes of x and y do not match')

#         self.model = model
#         self.name = name
#         if parameters is None:
# #             from ..utility.func_tools import get_func_params
#             self.parameters = get_func_params(model, exclude_x=True)
#         else:
#             self.parameters = list(parameters)
#         self.config = DictToAttr({
#             'opt_method': opt_method,
#             'exclude_zero': exclude_zero,
#             'init_guess': init_guess,
#             'rnd_seed': rnd_seed
#         })

#         if isinstance(x_data, list):
#             x_data = np.array(x_data)
#         if isinstance(y_data, list):
#             y_data = np.array(y_data)
#         if exclude_zero is True:
#             mask = y_data != 0
#         else:
#             mask = np.repeat(True, x_data.shape[0])

#         self.x_data = x_data[mask]
#         self.y_data = y_data[mask]
#         if weights is None:
#             weights = np.ones(len(y_data))
#         self.weights = weights[mask]

#         if bounds is None:
#             self.config.bounds = [np.repeat(-np.inf, len(self.parameters)),
#                                   np.repeat(np.inf, len(self.parameters))]
#         else:
#             self.config.bounds = bounds

#         if bootstrap_num > 0 and len(self.x_data) > 1:
#             if bs_return_num is None:
#                 bs_return_num = 0
#             elif bs_return_num < 0:
#                 bs_return_num = bootstrap_num
#             else:
#                 bs_return_num = bs_return_num
#             self.bootstrap = Bootstrap(fitter=self, bootstrap_num=bootstrap_num, return_num=bs_return_num,
#                                                method=bs_method, **kwargs)
#         else:
#             self.bootstrap = None

#         self.metrics = metrics
#         self.results = None
#         from .visualizer import fitting_curve_plot, bootstrap_params_dist_plot
#         from ..utility.func_tools import FuncToMethod
#         self.visualizer = FuncToMethod(obj=self, functions=[fitting_curve_plot,
#                                                             bootstrap_params_dist_plot])

    def _fit(self, model=None, x_data=None, y_data=None, weights=None, parameters=None, bounds=None,
             metrics=None, init_guess=None, opt_method=None):
        """perform fitting"""
        from scipy.optimize import curve_fit
        import numpy as np
        import pandas as pd
        import warnings

        if model is None:
            model = self.model
            parameters = self.parameters
        if x_data is None:
            x_data = self.x_data
        if y_data is None:
            y_data = self.y_data
        if weights is None:
            weights = self.weights
        if parameters is None:
            from ..utility.func_tools import get_func_params
            parameters = get_func_params(model, exclude_x=True)
        if bounds is None:
            bounds = self.config.bounds
        if metrics is None:
            metrics = self.metrics
        if init_guess is None:
            init_guess = self.config.init_guess
        if opt_method is None:
            opt_method = self.config.opt_method

        try:
            if init_guess is None:
                # use a random guess
                init_guess = [np.random.random() for _ in parameters]
            params, pcov = curve_fit(f=model,
                                     xdata=x_data, ydata=y_data,
                                     sigma=weights, method=opt_method,
                                     bounds=bounds, p0=init_guess)
            if metrics is not None:
                metrics_res = {name: fn(params) for name, fn in metrics.items()}
            else:
                metrics_res = None
        except RuntimeError:
            warnings.warn(
                f"RuntimeError for fitting model {self.model} on {self.name if self.name is not None else '*'} when\n"
                f'\tx = {self.x_data}\n'
                f'\ty={self.y_data}\n'
            )
            params = np.full(fill_value=np.nan, shape=len(parameters))
            pcov = np.full(fill_value=np.nan, shape=(len(parameters), len(parameters)))
            if metrics is not None:
                metrics_res = {name: np.nan for name, fn in metrics.items()}
            else:
                metrics_res = None
        return {
            'params': params,
            'pcov': pcov,
            'metrics': metrics_res
        }

    def fit(self):
        """Wrapper on _fit method"""

        import numpy as np
        import pandas as pd
        from ..utility.func_tools import DictToAttr

        if self.config.rnd_seed is not None:
            np.random.seed(self.config.rnd_seed)

        self.results = FitResults(fitter=self)
        point_est = self._fit()
        params = {key: value for key, value in zip(self.parameters, point_est['params'])}
        if point_est['metrics'] is not None:
            params.update(point_est['metrics'])
        self.results.point_estimation.params = pd.Series(params)
        self.results.point_estimation.pcov = pd.DataFrame(data=point_est['pcov'],
                                                          index=self.parameters, columns=self.parameters)
        if self.bootstrap is None:
            pass
        else:
            self.bootstrap.run()

    def summary(self, stats_included=None):
        return self.results.to_series(stats_included=stats_included)

    @classmethod
    def from_table(cls, table, seq, model, x_data, weights=None, bounds=None, bootstrap_num=0, bs_return_num=None, bs_method='pct_res', exclude_zero=False, init_guess=None, metrics=None, rnd_seed=None, **kwargs):
        """Get data from a row of `pd.DataFrame` table. `SeqTable` is not supported due to multiple tables contained"""
        import numpy as np
        import pandas as pd

        if isinstance(x_data, (list, np.ndarray)):
            x_data = pd.Series(x_data, index=table.columns)
        elif isinstance(x_data, pd.Series):
            x_data = x_data[table.columns]

        y_data = table.loc[seq]
        return cls(x_data=x_data, y_data=y_data, model=model, name=seq,
                   weights=weights, bounds=bounds,
                   bootstrap_num=bootstrap_num, bs_return_num=bs_return_num, bs_method=bs_method,
                   exclude_zero=exclude_zero, init_guess=init_guess, metrics=metrics, rnd_seed=rnd_seed, **kwargs)

    def load_results(self, json_o_path):
        self.results = FitResults(fitter=self)
        self.results.load_json(json_o_path)

In [ ]:
def __init__(self, x_data, y_data, model, name=None, parameters=None, weights=None, bounds=None, opt_method='trf',
                 bootstrap_num=0, bs_return_num=None, bs_method='pct_res',
                 exclude_zero=False, init_guess=None, metrics=None, rnd_seed=None, **kwargs):
        """

        Args:
            x_data (list, np.ndarray, pd.Series): list object will convert to np.ndarray
            y_data (list, np.ndarray, pd.Series):
            model:
            name:
            weights:
            bounds:
            opt_method:
            bootstrap_num:
            bs_return_num:
            bs_method:
            exclude_zero:
            init_guess:
            metrics:
            rnd_seed:
            **kwargs:
        """
        import numpy as np
        from k_seq.utility.func_tools import DictToAttr, get_func_params


        if len(x_data) != len(y_data):
            raise ValueError('Shapes of x and y do not match')

        self.model = model
        self.name = name
        if parameters is None:
#             from ..utility.func_tools import get_func_params
            self.parameters = get_func_params(model, exclude_x=True)
        else:
            self.parameters = list(parameters)
        self.config = DictToAttr({
            'opt_method': opt_method,
            'exclude_zero': exclude_zero,
            'init_guess': init_guess,
            'rnd_seed': rnd_seed
        })

        if isinstance(x_data, list):
            x_data = np.array(x_data)
        if isinstance(y_data, list):
            y_data = np.array(y_data)
        if exclude_zero is True:
            mask = y_data != 0
        else:
            mask = np.repeat(True, x_data.shape[0])

        self.x_data = x_data[mask]
        self.y_data = y_data[mask]
        if weights is None:
            weights = np.ones(len(y_data))
        self.weights = weights[mask]

        if bounds is None:
            self.config.bounds = [np.repeat(-np.inf, len(self.parameters)),
                                  np.repeat(np.inf, len(self.parameters))]
        else:
            self.config.bounds = bounds

        if bootstrap_num > 0 and len(self.x_data) > 1:
            if bs_return_num is None:
                bs_return_num = 0
            elif bs_return_num < 0:
                bs_return_num = bootstrap_num
            else:
                bs_return_num = bs_return_num
            self.bootstrap = Bootstrap(fitter=self, bootstrap_num=bootstrap_num, return_num=bs_return_num,
                                               method=bs_method, **kwargs)
        else:
            self.bootstrap = None

        self.metrics = metrics
        self.results = None
        from .visualizer import fitting_curve_plot, bootstrap_params_dist_plot
        from ..utility.func_tools import FuncToMethod
        self.visualizer = FuncToMethod(obj=self, functions=[fitting_curve_plot,
                                                            bootstrap_params_dist_plot])

In [82]:
def get_workers(seq):
    
    return least_square.SingleFitter.from_table(
        table=seq_table.reacted_frac_filtered,
        seq=seq,
        x_data=seq_table.x_values,
        model=BYOModel.func_react_frac_no_slope,
        parameters=['k', 'A'],
    )


%timeit worker_list = seq_table.reacted_frac_filtered.index[:100].to_series().apply(get_workers)

629 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
worker_list = seq_table.reacted_frac_filtered.index[:100].to_series().apply(get_workers)

In [84]:
%timeit [worker.fit() for worker in worker_list]

4.5 s ± 50.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Test standalone batch fitter


In [ ]:
class BatchFitter:

    def __str__(self):
        return 'Least squared BatchFitter'

    def __init__(self, table, x_values, model, weights=None, bounds=None, seq_to_fit=None,
                 bootstrap_num=0, bs_return_num=None, bs_method='pct_res',
                 opt_method='trf', exclude_zero=False, init_guess=None, metrics=None, rnd_seed=None,
                 save_single_fitters=True, **kwargs):
        from ..utility.func_tools import get_func_params, DictToAttr
        import pandas as pd
        import numpy as np

        self.model = model
        self.parameters = get_func_params(model, exclude_x=True)
        self.config = DictToAttr({
            'seq_to_fit': seq_to_fit,
            'save_single_fitters': save_single_fitters
        })
        self.fitters = None
        # prep fitting params once to save time for each single fitting
        self.x_values = x_values[table.columns.values]
        if weights is None:
            weights = np.ones(len(self.x_values))
        if bounds is None:
            bounds = [np.repeat(-np.inf, len(self.parameters)),
                      np.repeat(np.inf, len(self.parameters))]
        if len(x_values) <= 1:
            bootstrap_num = 0
        if bs_return_num is None:
            bs_return_num = 0
        elif bs_return_num < 0:
            bs_return_num = bootstrap_num
        else:
            pass
        if bs_return_num > 0:
            self.config.bootstrap = True

        # contains parameters should pass to the single fitter
        self.fit_params = DictToAttr({
            'x_data': self.x_values,
            'model': self.model,
            'parameters': self.parameters,
            'weights': weights,
            'bounds': bounds,
            'opt_method': opt_method,
            'bootstrap_num': bootstrap_num,
            'bs_return_num': bs_return_num,
            'bs_method': bs_method,
            'exclude_zero': exclude_zero,
            'init_guess': init_guess,
            'metrics': metrics,
            'rnd_seed': rnd_seed
        })

        if isinstance(table, str):
            from pathlib import Path
            table_path = Path(table)
            if table_path.is_file():
                try:
                    table = pd.read_pickle(table_path)
                except:
                    raise TypeError(f'{table_path} is not pickled pd.DataFrame object')
            else:
                raise FileNotFoundError(f'{table_path} is not a valid file')
        elif not isinstance(table, pd.DataFrame):
            raise TypeError('Table should be a pd.DataFrame')
        else:
            pass

        self.table = table
        if seq_to_fit is None:
            self.seq_list = self.table.index.values
        else:
            if isinstance(seq_to_fit, (list, np.ndarray, pd.Series)):
                self.seq_list = list(seq_to_fit)
            else:
                raise TypeError('Unknown seq_to_fit type, is it list-like?')

        self.results = BatchFitResults(fitter=self)

        # from .visualizer import fitting_curve_plot, bootstrap_params_dist_plot, param_value_plot
        # from ..utility import FunctionWrapper
        # self.visualizer = FunctionWrapper(data=self,
        #                                   functions=[
        #                                       fitting_curve_plot,
        #                                       bootstrap_params_dist_plot,
        #                                       param_value_plot
        #                                   ])

    def worker_generator(self):
        for seq in self.seq_list:
            try:
                yield SingleFitter.from_table(table=self.table, seq=seq, **self.fit_params.__dict__)
            except:
                raise Exception(f'Can not create fitting worker for {seq}')

    def fit(self, deduplicate=False, parallel_cores=1, dry_run=False):
        if deduplicate:
            self._hash()
        if dry_run:
            self.fitters = {worker.name: worker for worker in self.worker_generator()}
        else:
            if parallel_cores > 1:
                import multiprocessing as mp
                pool = mp.Pool(processes=int(parallel_cores))
                if self.config.save_single_fitters:
                    self.fitters = {seq[0]: seq[1] for seq in pool.map(_work_fn, self.worker_generator())}
                    self.results.update(self.fitters)
                else:
                    self.results.update(pool.map(_work_fn, self.worker_generator()))
            else:
                # single thread
                if self.config.save_single_fitters:
                    self.fitters = {seq[0]: seq[1] for seq in [_work_fn(fitter) for fitter in self.worker_generator()]}
                    self.results.update(self.fitters)
                else:
                    self.results.update([_work_fn(fitter) for fitter in self.worker_generator()])
            if deduplicate:
                self._hash_inv()

    @classmethod
    def from_SeqTable(cls, seq_table, model, seq_to_fit=None, weights=None, bounds=None, bootstrap_depth=0, bs_return_size=None,
                      resample_pct_res=False, missing_data_as_zero=False, random_init=True, metrics=None, **kwargs):
        if seq_to_fit is None:
            seq_to_fit = seq_table.reacted_frac_table.index
        if weights is None:
            weights = None
        if bounds is None:
            bounds = None
        if bs_return_size is None:
            bs_return_size = None
        if metrics is None:
            metrics = None
        return cls(seq_to_fit = {seq: seq_table.reacted_frac_table.loc[seq] for seq in seq_to_fit},
                   x_values=seq_table.x_values, model=model,
                   weights=weights, bounds=bounds,
                   bootstrap_depth=bootstrap_depth, bs_return_size=bs_return_size,
                   resample_pct_res=resample_pct_res, missing_data_as_zero=missing_data_as_zero,
                   random_init=random_init, metrics=metrics, **kwargs)

    def summary(self, save_to=None):
        if save_to is None:
            return self.results.summary()
        else:
            self.results.summary().to_csv(save_to)

    def _hash(self):
        """De-duplicate rows to avoid unnecessary fitting"""

        def hash_series(row):
            return hash(tuple(row))

        self._table_dup = self.table.copy()
        self.table = self.table.loc[self.seq_list]
        print(f'Shrink rows in table by removing duplicates: {self.table.shape[0]} -->', end='')
        self._seq_to_hash = self.table.apply(hash_series, axis=1).to_dict()
        self.table = self.table[~self.table.duplicated(keep='first')]
        self.table.rename(index=self._seq_to_hash, inplace=True)
        self._seq_list_dup = self.seq_list.copy()
        self.seq_list = [self._seq_to_hash[seq] for seq in self.seq_list]
        print(self.table.shape[0])

    def _hash_inv(self):
        """Recover the hashed results"""
        import pandas as pd
        print('Recovering original table...')
        self.table = self._table_dup.copy()
        del self._table_dup
        self.seq_list = self._seq_list_dup.copy()
        del self._seq_list_dup
        self.results.results = {seq: self.results.results[seq_hash] for seq, seq_hash in self._seq_to_hash.items()}


def _work_fn(worker):
    worker.fit()
    return worker.name, worker